# INTEGRATED USE OF MULTISOURCE REMOTE SENSING DATA FOR NATIONAL SCALE AGRICULTURAL  DROUGHT MONITORING IN KENYA
# ADM-Kenya, 2024
# Sentinel-2 Spectral/Vegetation Indexes Export - High-Resolution Evaportranspiration Estimation

Here, we want to load Sentinel-2 data collection. Cloud masking, generating VIs, and getting data over cropland pixels are like steps of crop condition mapping. Note that here we want to have daily products.

# Tools and Libraries

We will use the following libraries:

    openEO: A library to initiate using of openEO facility, 
        including various dependencies for processing data collections.
    NumPy: A fundamental package for scientific computing in Python.

In [10]:
import openeo
import openeo.extra.spectral_indices as opc 
import numpy as np
import openeo.processes as eop

In [11]:
connection = openeo.connect("openeo.dataspace.copernicus.eu").authenticate_oidc()

Authenticated using refresh token.


To prepare a Sentinel-2 image collection, it’s needed to assign data collection name (SENTINEL2_L2A), spatial extent, temporal extent, essential bands, and maximum percent of cloud cover per image (max_cloud_cover). Finally, a datacube is generated for desired location, period, and bands.

In [3]:
S2_2021_cloudfree = connection.load_collection(
  "SENTINEL2_L2A",
  spatial_extent={"west": 34.382, "south": 0.758, "east": 34.447, "north": 0.769},
  temporal_extent=["2021-04-01", "2021-04-30"],
  bands=["B02","B03","B04","B05","B06","B08","B11","B12","SCL"],
  max_cloud_cover=30
)

The next step is to mask pixels covered by clouds and clouds’ shadow in all three datacubes. This information is available in SCL band, accessing by band numbers, such as 3, 8, and 9.

In [4]:
scl_band_2021 = S2_2021_cloudfree.band("SCL")
cloud_mask_2021 = (scl_band_2021 == 3) | (scl_band_2021 == 8) | (scl_band_2021 == 9)

In [5]:
cloud_mask_2021 = cloud_mask_2021.resample_cube_spatial(S2_2021_cloudfree)
cube_masked_2021 = S2_2021_cloudfree.mask(cloud_mask_2021)

After masking the pixels covered by cloud, 5 spectral indices including NDVI, NDMI, GNDVI, EVI, and NMDI, generated by Sentinel-2 images at 20m spatial resolution (Sentinel-2_VIs.ipynb).

In [6]:
# Spectral indexes:
idx_list = ["NDVI","NDMI","GNDVI","NMDI","EVI"]
S2_VIs_2021 = opc.compute_indices(cube_masked_2021, idx_list).filter_bands(idx_list)

In [7]:
# Croplands masking:
cropland_mask = connection.load_collection("ESA_WORLDCOVER_10M_2021_V2", bands=["MAP"],
                                temporal_extent = ["2021-01-01", "2022-01-01"],
                                spatial_extent={"west": 34.282, "south": 0.658, "east": 34.447, "north": 0.769}
                    )

cropland_mask = cropland_mask.band("MAP") != 40
S2_2021_input_crop = S2_VIs_2021.mask(cropland_mask.max_time())

This platform lets users downloading images in NetCDF and GeoTIFF formats. In the case of Sentinel-2, it is more useful to download them as GeoTIFF to utilize them later in the downscaling procedure.

In [8]:
job = S2_2021_input_crop.create_job(out_format="GTiff")

In [ ]:
job.start_and_wait()
job.get_results()